## 환경 설정

In [11]:
import json
import os
from collections import Counter
from pathlib import Path
from typing import Any, Dict, List

import yaml
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from openrouter_llm import create_embedding_model, create_openrouter_llm

load_dotenv()

llm = create_openrouter_llm(model="gpt-4.1", temperature=0.3)
embeddings = create_embedding_model(model="text-embedding-3-small")

## 2. Persona 정의 로드

In [10]:
# Persona 정의 로드
persona_path = Path("persona_definitions.yaml")

with persona_path.open("r", encoding="utf-8") as f:
    persona_config = yaml.safe_load(f)

personas = persona_config["personas"]

print(f"{len(personas)}개 Persona 로드 완료:\n")
for p in personas:
    print(f"  - {p['name']}: {p['role_description'][:60]}...")

7개 Persona 로드 완료:

  - Junior Developer: 신입 또는 경력 1-2년차 개발자. 기본적인 기술 지식은 있지만 실무 경험이 부족하여 자세한 가이드가 필요함...
  - Senior Engineer: 경력 5년 이상의 숙련된 엔지니어. 깊은 기술 지식을 보유하고 있으며 간결하고 효율적인 커뮤니케이션을 선호함...
  - Non-Technical Manager: 기술 배경이 없는 팀장/관리자. 비즈니스 영향도와 결과 중심으로 사고하며 쉬운 설명을 필요로 함...
  - New Employee: 회사에 처음 입사한 신입 직원. 회사 시스템과 프로세스에 대한 지식이 전무하여 기본적인 것부터 질문함...
  - Power User: 시스템과 도구를 능숙하게 사용하는 중급 이상 사용자. 효율성과 자동화에 관심이 많으며 고급 기능을 적극적으로...
  - Remote Worker: 재택근무나 원격지에서 근무하는 직원. 네트워크 연결과 원격 접속 관련 이슈가 많으며 즉각적인 해결이 필요함...
  - Security-Conscious User: 보안과 개인정보 보호에 민감한 사용자. 모든 작업에서 보안 영향도를 고려하며 규정 준수를 중요하게 생각함...


## 3. Persona별 질문 생성

각 Persona의 특성을 반영한 질문을 생성합니다.

In [12]:
# 질문 생성 프롬프트 템플릿
question_generation_template = """
당신은 IT Helpdesk 시스템의 질문을 생성하는 전문가입니다.

다음 Persona의 특성을 정확히 반영하여 질문을 생성하세요:

**Persona**: {persona_name}
**역할**: {role_description}
**전문성 수준**: {expertise_level}
**기술 깊이**: {technical_depth}

**행동 패턴**:
{behaviors}

**질문 패턴 예시**:
{question_patterns}

**기존 질문 예시**:
{example_questions}

위 Persona의 특성을 반영하여 IT Helpdesk 관련 질문을 **{num_questions}개** 생성하세요.

조건:
1. 각 질문은 Persona의 행동 패턴과 커뮤니케이션 스타일을 명확히 반영
2. 질문 간 중복 최소화 (다양한 주제)
3. 실제 사용자가 물어볼 법한 현실적인 질문
4. 질문만 생성 (답변 불필요)

출력 형식: JSON 배열
```json
[
  {{"question": "질문 1"}},
  {{"question": "질문 2"}},
  ...
]
```
"""

question_prompt = ChatPromptTemplate.from_template(question_generation_template)

In [13]:
# Persona별 질문 생성 함수
def generate_questions_for_persona(persona: Dict, num_questions: int = 20) -> List[str]:
    """특정 Persona에 맞는 질문 생성"""

    # 프롬프트 구성
    behaviors_text = "\n".join([f"- {b}" for b in persona["behaviors"]])
    patterns_text = "\n".join([f"- {p}" for p in persona["question_patterns"]])
    examples_text = "\n".join([f"- {e}" for e in persona["example_questions"]])

    messages = question_prompt.format_messages(
        persona_name=persona["name"],
        role_description=persona["role_description"],
        expertise_level=persona["characteristics"]["expertise_level"],
        technical_depth=persona["characteristics"]["technical_depth"],
        behaviors=behaviors_text,
        question_patterns=patterns_text,
        example_questions=examples_text,
        num_questions=num_questions,
    )

    # LLM 호출
    response = llm.invoke(messages)

    # JSON 파싱
    try:
        # 응답에서 JSON 추출 (```json ... ``` 형식 처리)
        content = response.content
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0]
        elif "```" in content:
            content = content.split("```")[1].split("```")[0]

        questions_data = json.loads(content.strip())
        questions = [q["question"] for q in questions_data]
        return questions
    except Exception as e:
        print(f"JSON 파싱 오류: {e}")
        print(f"응답 내용: {response.content[:200]}...")
        return []

In [14]:
# 모든 Persona에 대해 질문 생성
all_persona_questions = {}

for i, persona in enumerate(personas):
    persona_name = persona["name"]
    print(f"[{i + 1}/{len(personas)}] {persona_name}...")

    questions = generate_questions_for_persona(persona, num_questions=20)
    all_persona_questions[persona_name] = questions

    print(f"{len(questions)}개 질문 생성 완료\n")

total_questions = sum(len(qs) for qs in all_persona_questions.values())
print(f"\n전체 {total_questions}개 질문 생성 완료")

[1/7] Junior Developer...
20개 질문 생성 완료

[2/7] Senior Engineer...
20개 질문 생성 완료

[3/7] Non-Technical Manager...
20개 질문 생성 완료

[4/7] New Employee...
20개 질문 생성 완료

[5/7] Power User...
20개 질문 생성 완료

[6/7] Remote Worker...
20개 질문 생성 완료

[7/7] Security-Conscious User...
20개 질문 생성 완료


전체 140개 질문 생성 완료


## 다양성 분석

Persona 간 질문 다양성을 임베딩 기반으로 분석합니다.

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 모든 질문 임베딩 생성
all_questions_flat = []
question_to_persona = {}

for persona_name, questions in all_persona_questions.items():
    for q in questions:
        all_questions_flat.append(q)
        question_to_persona[q] = persona_name

# 배치로 임베딩 생성
question_embeddings = embeddings.embed_documents(all_questions_flat)

print(f"{len(question_embeddings)}개 질문 임베딩 완료")

  0%|          | 0/1 [00:00<?, ?it/s]

140개 질문 임베딩 완료


In [16]:
# Persona 간 평균 유사도 계산
persona_avg_embeddings = {}

for persona_name, questions in all_persona_questions.items():
    persona_indices = [
        i for i, q in enumerate(all_questions_flat) if question_to_persona[q] == persona_name
    ]
    persona_embs = np.array([question_embeddings[i] for i in persona_indices])
    persona_avg_embeddings[persona_name] = persona_embs.mean(axis=0)

# Persona 간 코사인 유사도
persona_names = list(persona_avg_embeddings.keys())
persona_emb_matrix = np.array([persona_avg_embeddings[name] for name in persona_names])
similarity_matrix = cosine_similarity(persona_emb_matrix)

print("\nPersona 간 코사인 유사도 매트릭스:\n")
print(f"{'':20s}", end="")
for name in persona_names:
    print(f"{name[:15]:>18s}", end="")
print()

for i, name1 in enumerate(persona_names):
    print(f"{name1:20s}", end="")
    for j, name2 in enumerate(persona_names):
        print(f"{similarity_matrix[i][j]:18.3f}", end="")
    print()


Persona 간 코사인 유사도 매트릭스:

                       Junior Develope   Senior Engineer   Non-Technical M      New Employee        Power User     Remote Worker   Security-Consci
Junior Developer                 1.000             0.669             0.750             0.796             0.701             0.758             0.662
Senior Engineer                  0.669             1.000             0.633             0.570             0.624             0.668             0.604
Non-Technical Manager             0.750             0.633             1.000             0.795             0.767             0.833             0.746
New Employee                     0.796             0.570             0.795             1.000             0.734             0.728             0.713
Power User                       0.701             0.624             0.767             0.734             1.000             0.694             0.718
Remote Worker                    0.758             0.668             0.833             0.72

In [17]:
# 다양성 점수 (1 - 평균 유사도)
# 대각선 제외한 평균
off_diagonal = []
for i in range(len(similarity_matrix)):
    for j in range(len(similarity_matrix)):
        if i != j:
            off_diagonal.append(similarity_matrix[i][j])

avg_similarity = np.mean(off_diagonal)
diversity_score = 1 - avg_similarity

print("\n다양성 분석 결과:")
print(f"  - Persona 간 평균 유사도: {avg_similarity:.3f}")
print(f"  - 다양성 점수: {diversity_score:.3f}")
print("\n해석:")
if diversity_score > 0.3:
    print("  매우 다양한 질문 생성 (우수)")
elif diversity_score > 0.15:
    print("  적당한 다양성 (개선 가능)")
else:
    print("  낮은 다양성 (Persona 정의 개선 필요)")


다양성 분석 결과:
  - Persona 간 평균 유사도: 0.706
  - 다양성 점수: 0.294

해석:
  적당한 다양성 (개선 가능)


## 5. 동일 주제 Persona별 비교

동일한 주제(예: 비밀번호 재설정)에 대한 Persona별 질문 차이를 확인합니다.

In [18]:
# 주제: 비밀번호 재설정
topic_keyword = "비밀번호"

print(f"🔍 '{topic_keyword}' 관련 Persona별 질문:\n")

for persona_name, questions in all_persona_questions.items():
    related_questions = [q for q in questions if topic_keyword in q]
    if related_questions:
        print(f"━━━ {persona_name} ━━━")
        for q in related_questions[:2]:
            print(f"  • {q}")
        print()

🔍 '비밀번호' 관련 Persona별 질문:

━━━ Junior Developer ━━━
  • 회사에서 제공하는 원격 데스크톱 접속 아이디와 비밀번호를 분실했습니다. 복구 절차가 어떻게 되는지 안내해주실 수 있나요?

━━━ New Employee ━━━
  • PC 비밀번호를 잊어버렸는데 어떻게 초기화할 수 있는지, 그리고 절차가 어디 안내되어 있는지 알 수 있을까요?

━━━ Remote Worker ━━━
  • 지금 사내 그룹웨어 로그인 오류가 반복됩니다. 비밀번호 리셋도 해봤는데 안 됩니다. 오늘 업무에 꼭 필요합니다.

━━━ Security-Conscious User ━━━
  • 모든 비밀번호는 해시값으로 안전하게 저장되나요? 어떤 해시 알고리즘을 사용하나요?



## 6. 결과 저장

In [20]:
# JSONL 형식으로 저장
output_path = Path("persona_based_dataset.jsonl")

with output_path.open("w", encoding="utf-8") as f:
    for persona_name, questions in all_persona_questions.items():
        # Persona 정보 찾기
        persona_info = next((p for p in personas if p["name"] == persona_name), None)

        for question in questions:
            record = {
                "question": question,
                "persona": persona_name,
                "persona_characteristics": persona_info["characteristics"] if persona_info else {},
                "expected_behavior": {
                    "response_style": persona_info["characteristics"]["communication_style"]
                    if persona_info
                    else "Unknown",
                    "detail_level": persona_info["characteristics"]["detail_preference"]
                    if persona_info
                    else "Unknown",
                },
                "metadata": {"source": "persona_generator", "topic": "IT_Helpdesk"},
            }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"Persona 기반 데이터셋 저장 완료: {output_path}")
print(f" 총 {len(all_questions_flat)}개 레코드")

Persona 기반 데이터셋 저장 완료: persona_based_dataset.jsonl
 총 140개 레코드
